# بخش معتبرسنجی (Validation) در مدل‌های یادگیری ماشین با پایتون

In [1]:
# بخش معتبرسنجی (Validation) در مدل‌های یادگیری ماشین با پایتون

from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# تولید داده‌های نمونه برای تست
X, y = make_regression(n_samples=1000, n_features=20, noise=0.1, random_state=42)

# تقسیم داده‌ها
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# جستجوی شبکه‌ای برای تنظیم ابرپارامتر
params = {'n_estimators':[100,200,300],'max_depth':[5,10,15]}
model = GridSearchCV(RandomForestRegressor(random_state=42), params, cv=5, scoring='r2')
model.fit(X_train, y_train)

# ارزیابی مدل
y_pred = model.predict(X_test)
print("Best Parameters:", model.best_params_)
print("R2:", r2_score(y_test, y_pred))

# روش صحیح محاسبه RMSE بدون استفاده از پارامتر منسوخ شده squared
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", rmse)

Best Parameters: {'max_depth': 15, 'n_estimators': 300}
R2: 0.8243844930590914
RMSE: 82.44905791291929


In [2]:
# روش دوم: استفاده از root_mean_squared_error (در صورت موجود بودن)

try:
    from sklearn.metrics import root_mean_squared_error
    rmse = root_mean_squared_error(y_test, y_pred)
    print("RMSE (using root_mean_squared_error):", rmse)
except ImportError:
    # برای نسخه‌های قدیمی‌تر
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print("RMSE:", rmse)

RMSE (using root_mean_squared_error): 82.44905791291929


# بخش سمیت (Toxicity) و ADMET با پایتون

In [3]:
# ابتدا باید deepchem را نصب کنید
# در خط فرمان یا ترمینال اجرا کنید:
# pip install deepchem

import deepchem as dc
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# بخش سمیت (Toxicity) و ADMET با پایتون

try:
    # بارگذاری داده سمیت (Tox21 dataset)
    print("در حال بارگذاری داده‌های Tox21...")
    tasks, datasets, transformers = dc.molnet.load_tox21(featurizer='GraphConv')
    train_dataset, valid_dataset, test_dataset = datasets
    
    print(f"تعداد وظایف: {len(tasks)}")
    print(f"اندازه داده آموزشی: {len(train_dataset)}")
    print(f"اندازه داده اعتبارسنجی: {len(valid_dataset)}")
    print(f"اندازه داده تست: {len(test_dataset)}")
    
    # ساخت مدل یادگیری عمیق
    model = dc.models.GraphConvModel(n_tasks=len(tasks), mode='classification', 
                                   dropout=0.2, random_seed=42)
    
    # آموزش مدل
    print("شروع آموزش مدل...")
    model.fit(train_dataset, nb_epoch=10)  # تعداد epoch کاهش یافت برای اجرای سریع‌تر
    
    # ارزیابی مدل
    metrics = [dc.metrics.Metric(dc.metrics.roc_auc_score)]
    print("در حال ارزیابی مدل...")
    
    train_scores = model.evaluate(train_dataset, metrics, transformers)
    valid_scores = model.evaluate(valid_dataset, metrics, transformers)
    
    print(f"ROC-AUC آموزشی: {train_scores['mean-roc_auc_score']:.4f}")
    print(f"ROC-AUC اعتبارسنجی: {valid_scores['mean-roc_auc_score']:.4f}")
    
except ImportError:
    print("کتابخانه deepchem نصب نیست!")
    print("لطفاً با دستور زیر نصب کنید: pip install deepchem")
except Exception as e:
    print(f"خطا در اجرای کد: {e}")


AttributeError: module 'ml_dtypes' has no attribute 'float4_e2m1fn'

In [ ]:
# کد کامل و بهینه‌شده برای deepchem
def run_toxicity_analysis():
    try:
        import deepchem as dc
        import numpy as np
        
        print("=== آنالیز سمیت با DeepChem ===")
        
        # بارگذاری داده‌ها
        print("1. بارگذاری داده‌های Tox21...")
        tasks, datasets, transformers = dc.molnet.load_tox21(
            featurizer='GraphConv', 
            reload=False  # برای جلوگیری از دانلود مجدد
        )
        train_dataset, valid_dataset, test_dataset = datasets
        
        print(f"   - تعداد وظایف: {len(tasks)}")
        print(f"   - اندازه مجموعه‌ها: آموزش={len(train_dataset)}, اعتبارسنجی={len(valid_dataset)}, تست={len(test_dataset)}")
        
        # ساخت مدل
        print("2. ساخت مدل GraphConv...")
        model = dc.models.GraphConvModel(
            n_tasks=len(tasks),
            mode='classification',
            dropout=0.2,
            random_seed=42
        )
        
        # آموزش مدل
        print("3. آموزش مدل (10 دوره)...")
        model.fit(train_dataset, nb_epoch=10)
        
        # ارزیابی
        print("4. ارزیابی مدل...")
        metric = dc.metrics.Metric(dc.metrics.roc_auc_score)
        
        train_score = model.evaluate(train_dataset, [metric], transformers)
        valid_score = model.evaluate(valid_dataset, [metric], transformers)
        
        print("\n--- نتایج ---")
        print(f"ROC-AUC آموزشی: {train_score['mean-roc_auc_score']:.4f}")
        print(f"ROC-AUC اعتبارسنجی: {valid_score['mean-roc_auc_score']:.4f}")
        
        return model, datasets, transformers
        
    except ImportError:
        print("خطا: کتابخانه deepchem یافت نشد!")
        print("راه‌حل: pip install deepchem")
        return None
    except Exception as e:
        print(f"خطا در اجرا: {e}")
        return None

# اجرای تابع
if __name__ == "__main__":
    run_toxicity_analysis()

# 🧩 فاز ۱: آماده‌سازی محیط و داده‌ها

هدف: بارگذاری داده سمیت دارویی (مثل Tox21 dataset) که شامل ترکیبات شیمیایی و برچسب سمیت آنهاست.

In [ ]:
!pip install deepchem rdkit-pypi

import deepchem as dc
import numpy as np


# 🔹 بارگذاری داده‌ی سمیت:

In [ ]:
# بارگذاری داده سمیت از مجموعه Tox21
tasks, datasets, transformers = dc.molnet.load_tox21(featurizer='GraphConv')
train_dataset, valid_dataset, test_dataset = datasets

print("تعداد ویژگی‌ها:", train_dataset.X.shape)
print("تعداد نمونه‌ها:", len(train_dataset))
print("تعداد برچسب‌ها (سمیت‌ها):", len(tasks))


# نتیجه: داده‌های Tox21 شامل بیش از ۸۰۰۰ ترکیب است که سمیت آنها برای چندین مسیر سلولی مختلف مشخص شده است.

# 🧠 فاز ۲: ساخت مدل یادگیری عمیق برای پیش‌بینی سمیت

هدف: ایجاد یک مدل گراف کانولوشنی (Graph Convolutional Network) که ساختار شیمیایی مولکول را به‌صورت گرافی پردازش می‌کند

In [ ]:
# مدل یادگیری عمیق با معماری GraphConvModel
model = dc.models.GraphConvModel(
    n_tasks=len(tasks),        # تعداد برچسب‌های سمیت
    mode='classification',     # حالت طبقه‌بندی
    dropout=0.2,               # جلوگیری از overfitting
    model_dir='tox21_model'    # مسیر ذخیره مدل
)

# آموزش مدل
model.fit(train_dataset, nb_epoch=30)


# 🧪 فاز ۳: معتبرسنجی مدل (Validation)

هدف: بررسی عملکرد مدل در پیش‌بینی سمیت داروهای جدید با معیار ROC-AUC

In [ ]:
from deepchem.metrics import Metric, roc_auc_score

metric = Metric(roc_auc_score, np.mean)
train_score = model.evaluate(train_dataset, [metric])
valid_score = model.evaluate(valid_dataset, [metric])

print("Train ROC-AUC:", train_score)
print("Validation ROC-AUC:", valid_score)


# 💡 ROC-AUC بالا (مثلاً > 0.8) نشان‌دهنده قدرت پیش‌بینی خوب مدل در شناسایی ترکیبات سمی است.

# ⚙️ فاز ۴: تست روی داده جدید (External Validation)

می‌توانیم یک مولکول جدید را به‌صورت SMILES بدهیم و سمیتش را پیش‌بینی کنیم:

In [ ]:
from rdkit import Chem

# نمونه SMILES برای دارو (مثلاً caffeine)
mol = Chem.MolFromSmiles('Cn1cnc2c1c(=O)n(C)c(=O)n2C')
featurizer = dc.feat.ConvMolFeaturizer()
mol_feat = featurizer.featurize([mol])

# پیش‌بینی سمیت
prediction = model.predict_on_batch(mol_feat)
print("Predicted Toxicity Probabilities:", prediction)


# 📊 فاز ۵: اعتبارسنجی متقاطع و تنظیم ابرپارامترها (Cross-validation & Hyperparameter Tuning)

برای افزایش دقت و جلوگیری از overfitting:

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5)
scores = []

for train_idx, test_idx in kf.split(train_dataset.X):
    subset_train = train_dataset.select(train_idx)
    subset_test = train_dataset.select(test_idx)
    model.fit(subset_train, nb_epoch=15)
    result = model.evaluate(subset_test, [metric])
    scores.append(result['mean-roc_auc_score'])
    
print("میانگین AUC در Cross-validation:", np.mean(scores))


# ⚗️ فاز ۱: بارگذاری و آماده‌سازی داده‌های ADMET

کتابخانه‌ی DeepChem مجموعه‌ای از داده‌های استاندارد برای ADMET دارد، مثلاً Therapeutics Data Commons (TDC) یا ADMET dataset.

In [ ]:
!pip install deepchem rdkit-pypi pandas numpy

import deepchem as dc
import pandas as pd
import numpy as np


# 🔹 بارگذاری داده‌ها:

In [ ]:
tasks, datasets, transformers = dc.molnet.load_admet(featurizer='GraphConv')
train_dataset, valid_dataset, test_dataset = datasets

print("تعداد ویژگی‌های مولکولی:", train_dataset.X.shape)
print("تعداد تسک‌ها (ویژگی‌های ADMET):", len(tasks))
print("نام تسک‌ها:", tasks)


# 📌 این داده شامل برچسب‌هایی برای چند خاصیت زیستی مانند:

جذب در روده (Human Intestinal Absorption)

نفوذپذیری سلولی (Caco-2 Permeability)

متابولیسم توسط CYP450

سمیت (Hepatotoxicity, hERG inhibition)

# 🧠 فاز ۲: ساخت مدل چندوظیفه‌ای (Multi-task GraphConv)

در اینجا مدل یادگیری عمیقی می‌سازیم که همه‌ی ویژگی‌های ADMET را به طور هم‌زمان پیش‌بینی کند.

In [ ]:
# مدل چندوظیفه‌ای برای ADMET
model = dc.models.GraphConvModel(
    n_tasks=len(tasks),
    mode='regression',  # چون اغلب مقادیر ADMET پیوسته هستند
    dropout=0.2,
    model_dir='admet_multitask_model'
)

model.fit(train_dataset, nb_epoch=40)


# 📘 یادگیری چندوظیفه‌ای به مدل کمک می‌کند روابط بین ویژگی‌ها (مثل ارتباط سمیت با متابولیسم) را یاد بگیرد — درست همان چیزی که مقاله هم به آن اشاره داشت.

# 🧪 فاز ۳: ارزیابی دقت مدل

برای سنجش مدل در پیش‌بینی مقادیر ADMET:

In [ ]:
from deepchem.metrics import Metric, mean_absolute_error

metric = Metric(mean_absolute_error, np.mean)

train_score = model.evaluate(train_dataset, [metric])
valid_score = model.evaluate(valid_dataset, [metric])

print("Train MAE:", train_score)
print("Validation MAE:", valid_score)


# 
💡 MAE پایین‌تر → پیش‌بینی دقیق‌تر ADMET

# 🧩 فاز ۴: پیش‌بینی ویژگی‌های ADMET برای مولکول جدید

فرض کنید می‌خواهیم رفتار داروی Aspirin را پیش‌بینی کنیم:

In [ ]:
from rdkit import Chem

smiles = 'CC(=O)OC1=CC=CC=C1C(=O)O'  # Aspirin
mol = Chem.MolFromSmiles(smiles)
featurizer = dc.feat.ConvMolFeaturizer()
mol_feat = featurizer.featurize([mol])

prediction = model.predict_on_batch(mol_feat)
df = pd.DataFrame(prediction, columns=tasks)
print(df.T)


# ⚙️ فاز ۵: اعتبارسنجی خارجی (External Validation)

برای اطمینان از تعمیم‌پذیری مدل، روی مجموعه‌ای از ترکیبات ناشناخته آزمایش می‌شود:

In [ ]:
metrics = [dc.metrics.Metric(dc.metrics.pearson_r2_score, np.mean)]
test_score = model.evaluate(test_dataset, metrics)
print("External R² score:", test_score)


# 📊 فاز ۶: مصورسازی نتایج

می‌توانی برای نمایش در وبینار از نمودار پراکندگی یا اهمیت ویژگی‌ها استفاده کنی:

In [ ]:
import matplotlib.pyplot as plt

true = valid_dataset.y[:, 0]  # مثلاً جذب
pred = model.predict(valid_dataset)[:, 0]

plt.scatter(true, pred)
plt.xlabel("True Absorption")
plt.ylabel("Predicted Absorption")
plt.title("ADMET Prediction Validation")
plt.show()


# 🔬 فاز ۸: محاسبه همبستگی بین ویژگی‌های ADMET

ابتدا خروجی مدل (پیش‌بینی تمام ویژگی‌ها برای داده‌های تست) را می‌گیریم و در قالب یک جدول ذخیره می‌کنیم.

In [ ]:
preds = model.predict(test_dataset)
df_pred = pd.DataFrame(preds, columns=tasks)

# محاسبه ماتریس همبستگی
corr_matrix = df_pred.corr(method='pearson')
corr_matrix.round(2)


# 🌡️ فاز ۹: نمایش بصری با Heatmap

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8,6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
plt.title("ADMET Feature Correlation Heatmap", fontsize=14)
plt.show()


رنگ‌های قرمز: همبستگی مثبت (افزایش یک ویژگی باعث افزایش دیگری می‌شود)

رنگ‌های آبی: همبستگی منفی (افزایش یک ویژگی باعث کاهش دیگری می‌شود)

مربع‌های تیره‌تر → قدرت بیشتر رابطه

# 🧩 فاز ۱۰: ساخت شبکه همبستگی (Correlation Network)

نمایش شبکه‌ای روابط بین ویژگی‌های ADMET، به‌ویژه سمیت، برای ارائه‌ی بصری در وبینار بسیار جذاب است:

In [ ]:
import networkx as nx

# آستانه برای نمایش ارتباط‌های قوی‌تر
threshold = 0.5  
G = nx.Graph()

# افزودن یال‌ها بین ویژگی‌هایی که همبستگی بالایی دارند
for i in corr_matrix.columns:
    for j in corr_matrix.columns:
        if i != j and abs(corr_matrix.loc[i, j]) > threshold:
            G.add_edge(i, j, weight=corr_matrix.loc[i, j])

plt.figure(figsize=(7,7))
pos = nx.spring_layout(G, k=0.4)
nx.draw(G, pos, with_labels=True, node_color='skyblue', node_size=2000, font_size=12)
plt.title("ADMET Correlation Network", fontsize=14)
plt.show()
